In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    model_name="gpt-5-nano",
    temperature=1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie_title": "Inception",
        "answer": """
            Title: Inception
            Director: Christopher Nolan
            Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page
            Budget: $160 million
            Box Office: $836 million
            Genres: Science fiction, Action, Thriller
            Synopsis: A thief enters shared dreams to plant an idea and earn a chance to return to his family.
        """,
    },
    {
        "movie_title": "The Matrix",
        "answer": """
            Title: The Matrix
            Director: Lana Wachowski, Lilly Wachowski
            Main Cast: Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss
            Budget: $63 million
            Box Office: $466 million
            Genres: Science fiction, Action
            Synopsis: A hacker learns reality is a simulation and joins a rebellion to free humanity from intelligent machines.
        """,
    },
    {
        "movie_title": "Parasite",
        "answer": """
            Title: Parasite
            Director: Bong Joon-ho
            Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam
            Budget: $11 million (approx.)
            Box Office: $258 million (approx.)
            Genres: Drama, Thriller, Dark comedy
            Synopsis: A poor family slowly infiltrates a wealthy household, leading to unexpected violence and class conflict.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Tell me about the movie {movie_title}."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie information assistant. "
            "Given a movie title, you always reply using exactly this format:\n\n"
            "Title: <movie title>\n"
            "Director: <director(s)>\n"
            "Main Cast: <comma-separated main cast>\n"
            "Budget: <production budget>\n"
            "Box Office: <worldwide box office>\n"
            "Genres: <comma-separated genres>\n"
            "Synopsis: <one or two sentence summary>\n\n"
            "Do not add any other sections, comments, or text outside this format.",
        ),
        example_prompt,
        ("human", "Tell me about the movie {movie_title}."),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie_title": "Interstellar"})

Title: Interstellar
Director: Christopher Nolan
Main Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain, Michael Caine, Mackenzie Foy
Budget: $165 million
Box Office: $677 million
Genres: Science fiction, Adventure, Drama
Synopsis: A former NASA pilot joins a mission through a wormhole to seek a new home for humanity as Earth deteriorates. Time dilation and gravity challenges complicate the voyage.

AIMessageChunk(content='Title: Interstellar\nDirector: Christopher Nolan\nMain Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain, Michael Caine, Mackenzie Foy\nBudget: $165 million\nBox Office: $677 million\nGenres: Science fiction, Adventure, Drama\nSynopsis: A former NASA pilot joins a mission through a wormhole to seek a new home for humanity as Earth deteriorates. Time dilation and gravity challenges complicate the voyage.')